In [81]:
import csv
import pandas as pd
import igraph as ig
from igraph import *

In [82]:
data_file = '/home/dipp/Github/Master-Thesis-dipp/Network_On_Global_Scale/Data/sorted_country_code_with_count_new.csv'

In [83]:
country_data = {}
with open(data_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    count = 0
    for row in reader:
        if(row['i'] != 'None' and row['j'] != 'None'):
            if(row['i']!=row['j']):
                country_data[count] = [row['i'], row['j'], int(row['count'])]
                count+=1

In [84]:
df = pd.DataFrame.from_dict(country_data, orient='index', columns=['i','j','count'])

In [85]:
df

,i,j,count
0,'GB','US',261883
1,'US','GB',228077
2,'US','CA',136135
3,'IN','US',99852
4,'US','IN',88797
...,...,...,...
6303,'SN','AU',1
6304,'TT','NP',1
6305,'LC','US',1
6306,'IQ','JO',1


In [86]:
##Creating the Graph
g = Graph.DataFrame(df, directed=True, use_vids=False)

In [87]:
#tuples = [tuple(x) for x in df.values]
##g = Graph.TupleList(df, directed=False)
#g = Graph.TupleList(tuples, directed=False, edge_attrs=['count'])

In [88]:
#deg = g.degree(mode='all', loops=False)

In [89]:
deg = g.strength(mode='all', loops='True', weights='count')

In [90]:
degree_data_df = pd.DataFrame({attr: g.vs[attr] for attr in g.vertex_attributes()})

In [91]:
##This is the Node
degree_data_df['Degree'] = deg

In [92]:
degree_data_df

,name,Degree
0,'GB',714016.0
1,'US',1617816.0
2,'CA',259814.0
3,'IN',259796.0
4,'ZA',80877.0
...,...,...
213,'SC',3.0
214,'XK',1.0
215,'MH',1.0
216,'CV',1.0


In [93]:
degree_dict = {}
for index, row in degree_data_df.iterrows():
    degree_dict[row['name']] = row['Degree']

In [94]:
vi = []
vj = []
for index, row in df.iterrows():
    ai = row['i']
    bi = row['j']

    vi.append(degree_dict.get(ai))
    vj.append(degree_dict.get(bi))
    

In [95]:
df['vi'] = vi
df['vj'] = vj

In [96]:
df

,i,j,count,vi,vj
0,'GB','US',261883,714016.0,1617816.0
1,'US','GB',228077,1617816.0,714016.0
2,'US','CA',136135,1617816.0,259814.0
3,'IN','US',99852,259796.0,1617816.0
4,'US','IN',88797,1617816.0,259796.0
...,...,...,...,...,...
6303,'SN','AU',1,63.0,119320.0
6304,'TT','NP',1,359.0,2542.0
6305,'LC','US',1,34.0,1617816.0
6306,'IQ','JO',1,2899.0,780.0


In [97]:
cij = []
for index, row in df.iterrows():
    nij = row['count']
    nik = row['vi']
    njk = row['vj']

    temp_cij = nij / (nik + njk)

    cij.append(float(temp_cij))


In [98]:
df['cij'] = cij

In [99]:
df

,i,j,count,vi,vj,cij
0,'GB','US',261883,714016.0,1617816.0,1.123078e-01
1,'US','GB',228077,1617816.0,714016.0,9.781022e-02
2,'US','CA',136135,1617816.0,259814.0,7.250363e-02
3,'IN','US',99852,259796.0,1617816.0,5.318032e-02
4,'US','IN',88797,1617816.0,259796.0,4.729252e-02
...,...,...,...,...,...,...
6303,'SN','AU',1,63.0,119320.0,8.376402e-06
6304,'TT','NP',1,359.0,2542.0,3.447087e-04
6305,'LC','US',1,34.0,1617816.0,6.181043e-07
6306,'IQ','JO',1,2899.0,780.0,2.718130e-04


In [102]:
df.sort_values(by=['cij'], ascending=True)

,i,j,count,vi,vj,cij
5335,'SR','US',1,83.0,1617816.0,6.180856e-07
4605,'TL','US',1,80.0,1617816.0,6.180867e-07
4778,'LY','US',1,66.0,1617816.0,6.180920e-07
5751,'IO','US',1,53.0,1617816.0,6.180970e-07
5235,'BG','US',1,51.0,1617816.0,6.180978e-07
...,...,...,...,...,...,...
3,'IN','US',99852,259796.0,1617816.0,5.318032e-02
2,'US','CA',136135,1617816.0,259814.0,7.250363e-02
139,'KW','SA',1420,3949.0,11250.0,9.342720e-02
1,'US','GB',228077,1617816.0,714016.0,9.781022e-02


In [100]:
new_df = df.sort_values(by='cij')
new_df

,i,j,count,vi,vj,cij
5335,'SR','US',1,83.0,1617816.0,6.180856e-07
4605,'TL','US',1,80.0,1617816.0,6.180867e-07
4778,'LY','US',1,66.0,1617816.0,6.180920e-07
5751,'IO','US',1,53.0,1617816.0,6.180970e-07
5235,'BG','US',1,51.0,1617816.0,6.180978e-07
...,...,...,...,...,...,...
3,'IN','US',99852,259796.0,1617816.0,5.318032e-02
2,'US','CA',136135,1617816.0,259814.0,7.250363e-02
139,'KW','SA',1420,3949.0,11250.0,9.342720e-02
1,'US','GB',228077,1617816.0,714016.0,9.781022e-02
